In [1]:
import spacy
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [88]:
import pandas as pd

# load txt file into dataframe
df = pd.read_csv('generalized_swaps.txt', sep='\t', header=None, names=['col1', 'col2'])
# print first 5 rows of dataframe
sentence1 = [simple_preprocess(str(sentence)) for sentence in df['col1']]
sentence2 = [simple_preprocess(str(sentence)) for sentence in df['col2']]

In [3]:
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    new_text = []
    docs = nlp(text)
    words = nltk.word_tokenize(text)
    tokenizer = RegexpTokenizer(r'\w+')
    for token in docs:
        if not token.is_punct:
            txt = tokenizer.tokenize(token.text)
            filtered_word = [word for word in txt if word.lower() not in stop_words]
            new_text.extend(filtered_word)
    return ' '.join(new_text)

In [4]:
from gensim.models import Word2Vec

In [5]:
# from nltk import word_tokenize, sent_tokenize
# sentence = [word_tokenize(word) for word in sent_tokenize(sentence1)]
# sentence

In [110]:
import gensim.downloader as api
sentences = sentence1 + sentence2
model = api.load('word2vec-google-news-300')
# model = Word2Vec(sentences, min_count=1, vector_size=100, workers=4)

In [115]:
word_embed1 = [model[word] for word in sentence1]
word_embed2 = [model[word] for word in sentence2]

KeyError: "Key 'airwoman' not present"

In [ ]:
mean_col1 = np.mean(word_embed1, axis=0)
mean_col2 = np.mean(word_embed2, axis=0)

In [ ]:
gender_direction = mean_col1 - mean_col2

In [ ]:
gender_direction /= np.linalg.norm(gender_direction)

In [ ]:
word_embed_gender = {}
for word in model.key_to_index:
    embedding = model[word]
    dot_product = np.dot(embedding.reshape(1,-1), gender_direction.T)
    word_embed_gender[word] = dot_product[0]

In [113]:
from sklearn.metrics.pairwise import cosine_similarity

def analogy(word_a, word_b, word_c, model):
    embed_a = model[word_a]
    embed_b = model[word_b]
    embed_c = model[word_c]
    
    vec_d = embed_b - embed_a
    closest_word, closest_distance = None, float('inf')
    for word in model.key_to_index:
        if word in[word_a, word_b, word_c]:
            continue
        embedding = model[word].reshape(1,-1)
        analogy_vector = (embed_c - vec_d).reshape(1,-1)
        distance = cosine_similarity(embedding, analogy_vector)
        
        if distance < closest_distance:
            closest_word = word
            closest_distance = distance
    return closest_word

In [ ]:
word_a = 'man'
word_b = 'king'
word_c = 'woman'

closest_word = analogy(word_a, word_b, word_c, model)
print(f"{word_a} is to {word_b} as {word_c} is to {closest_word}")